In [4]:
import os
os.environ['https_proxy'] = "http://proxy-dmz.intel.com:912"

In [5]:
%pwd

'C:\\Users\\nanandas\\Cert Work\\Text-Summarization-NLP-Project-main\\Text-Summarization-NLP-Project-main\\research'

In [6]:
os.chdir("../")

In [7]:
%pwd

'C:\\Users\\nanandas\\Cert Work\\Text-Summarization-NLP-Project-main\\Text-Summarization-NLP-Project-main'

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [9]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [16]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

[2024-02-14 16:32:48,725: INFO: utils: NumExpr defaulting to 8 threads.]
[2024-02-14 16:32:51,412: INFO: config: PyTorch version 1.12.0 available.]
[2024-02-14 16:32:51,427: INFO: config: TensorFlow version 2.10.0 available.]


In [19]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [21]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    #data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-14 16:37:47,442: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-14 16:37:47,470: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-14 16:37:47,473: INFO: common: created directory at: artifacts]
[2024-02-14 16:37:47,475: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-14 16:37:47,477: INFO: 1434958058: File already exists of size: ~ 128 KB]
